In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/subjects/EmailSubjectTest.txt
/kaggle/input/subjects/EmailSubjectTrain.txt


In [2]:
 !pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 2.6 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [3]:
!pip install nltk==3.8.1 -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [4]:
!git clone https://github.com/ryanzhumich/AESLC

Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 17.32 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [5]:
import os
import re
import nltk
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

2024-07-29 09:33:59.339760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 09:33:59.339898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 09:33:59.643734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Function for text preprocessing
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalnum()]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [8]:
# Custom dataset class
class EmailSubjectDataset(Dataset):
    def __init__(self, emails, subjects, tokenizer, max_length):
        self.emails = emails
        self.subjects = subjects
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.emails)

    def __getitem__(self, idx):
        email_text = self.emails[idx]
        subject_text = self.subjects[idx]

        # Tokenize inputs and outputs
        inputs = self.tokenizer(email_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                padding='max_length')
        outputs = self.tokenizer(subject_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                 padding='max_length')

        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten(),
            'labels': outputs.input_ids.flatten()
        }

In [9]:
# Directory containing email files
emails_directory = '/kaggle/working/AESLC/enron_subject_line/train'

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [10]:
# Load emails and subjects from files
emails = []
subjects = []

for filename in os.listdir(emails_directory):

    with open(os.path.join(emails_directory, filename), 'r', encoding='utf-8', errors='ignore') as file:
        email_text = file.read()
        # Extract subject from email body using regex
        match = re.search(r'@subject\s+(.*)', email_text, re.IGNORECASE | re.DOTALL)

        if match:
            subject = match.group(1).strip()
            email_body = re.sub(r'@subject\n(.+)\n', '', email_text).strip()
            preprocessed_email = preprocess_text(email_body)

            emails.append(preprocessed_email)
            subjects.append(subject)

In [11]:
# Create dataset
print(emails[0])
print(subjects[0])
dataset = EmailSubjectDataset(emails, subjects, tokenizer, max_length=512)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

greeting folk heard ken lay met california ceo last friday back east result meeting mcnealy ceo sun asked background detail ken lay proposed solution california meeting california past couple week mcnealy others expressed willingness make phone call policymakers etc try advance ball pulled together hastily friday take responsibility error omission ramblings etc wanted distribute however make sure info getting distributed know folk busy likely get distributed ceo week comment suggestion etc improve welcome appreciated best subject backgrounder mcnealy
Backgrounder for McNealy


In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",
)

# Initialize model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Fine-tuning the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.096800,0.093909
2,0.090000,0.091731
3,0.086300,0.091299


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4332, training_loss=0.09385014020615118, metrics={'train_runtime': 3889.7176, 'train_samples_per_second': 8.907, 'train_steps_per_second': 1.114, 'total_flos': 9052201156608000.0, 'train_loss': 0.09385014020615118, 'epoch': 3.0})

In [13]:
# Function to generate subject lines from email text
def get_first_four_words_split(text):
    words = text.split()
    return ' '.join(words[:4])




def generate_subject(model, tokenizer, email_text, max_length=512):
    # Move model to correct device
    device = next(model.parameters()).device

    email_text = preprocess_text(email_text)

    input_ids = tokenizer.encode(email_text, return_tensors='pt').to(device)  # Move input_ids to device

    # Generate subject line
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)

    generated_subject = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_subject

example_email = "The following reports have been waiting for your approval for more than 4 days. Please review. Owner: James W Reitmeyer Report Name: JReitmeyer 10/24/01 Days In Mgr.Queue: 5"
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

example_email = "All, The below Analyst & Associate recruiting dates require ENA participation at Manager level at above. In order to resource each of your departments it is important to have ENA's involvement and participation in the interviews and debrief sessions on Fantastic Friday and Super Saturday events. These de-brief sessions will allow you the opportunity to select candidates you wish to join your groups. The target is to assign potential candidates to business units and departments from the outset. As ENA has the highest percentage of A&A rotating in its business unit, the participation of ENA at interview should reflect this. Therefore, please encourage your direct reports and managers to participate in the below events in order to secure candidates for your business area. Associate Recruiting: Saturday November 3 Total - 70 Candidates for Interview Analyst Recruiting: Friday, November 16 Total - 70 Candidates for Interview Associate Recruiting: Saturday, December 1 Total - 70 Candidates for Interview The above spreadsheet represents ENA's particpation today which I believe highlights the need for much additional support in these efforts. Please confirm by return participation of your respective groups. Regards,"
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

example_email = "Late on October 25th, we received information about a nonspecific threat to the Enron Center. We communicated with law enforcement officials who found the threat unsubstantiated and without merit. Nonetheless we take all threats seriously and have increased the security presence at the Enron Center still further. Once again, if you observe suspicious behavior, please call security at 3-6200."
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

example_email = "Thanks in advance for agreeing to speak at the Global Operations Controller Forum. There will be approximately 30 Enron business controllers present at the meeting. All have responsibility for mid and back office operations for the following Enron entities: Enron North America, Enron Europe, Enron South America, Enron Global Markets, Enron Industrial Markets, Enron Broadband Services and Enron Energy Services. Attendees will be here from Houston, Calgary, Tokyo, Sydney, London and New York (metals business). Attached for your reference is the agenda. There may be some slight changes before the forum begins, but this will give you a good idea of the topics to be covered and the other speakers who will address the group. You are scheduled to address the group as follows:"
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

example_email = "To confirm: Mark Thomas Mike Presley both previously on the list for Netco's Market Risk/Research Group have resigned. I'm combing the Estate for replacements. DP"
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

example_email = "Michelle; This is the presentation which was provided to the HR VP's. The HR VPs were tasked with going off to met their BU OTCs and devise plans around the three components. We are currently collating those plans into one document. We'll send that along to you when completed. Pls call if you have any questions."
generated_subject = generate_subject(model, tokenizer, example_email)
print("Generated Subject:", get_first_four_words_split(generated_subject))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask 

Generated Subject: following report waiting approval
Generated Subject: analyst associate recruiting date
Generated Subject: late october 25th received
Generated Subject: thanks advance agreeing speak
Generated Subject: confirm mark thomas mike


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Subject: michelle presentation provided hr


In [33]:
# Save model to local directory
local_model_dir = './saved_model'
trainer.save_model(local_model_dir)

In [30]:
print(os.listdir('/kaggle/working'))

['AESLC', 'wandb', 'results', 'gpt2model.zip', 'finalmodel.zip', 'logs', '.virtual_documents', 'saved_model']


In [31]:
#!zip -r /kaggle/working/finalmodel.zip /kaggle/working/saved_model

import shutil
import os

zip_filename = '/kaggle/working/finalmodel.zip'
shutil.make_archive(zip_filename.split('.')[0], 'zip', local_model_dir)

# Note: make_archive creates a ZIP file with the base name (finalmodel) in the current working directory (/kaggle/working/)

# Clean up: Remove the original saved model directory
shutil.rmtree(local_model_dir)

In [32]:
# Saving and downloading the model

import joblib
import zipfile

# Assuming 'model' is your trained model object
# Example: saving a trained model using joblib
joblib.dump(model, '/kaggle/working/model.pkl')

# Create a ZIP file containing the saved model
with zipfile.ZipFile('/kaggle/working/gpt2model.zip', 'w') as zipf:
    zipf.write('/kaggle/working/model.pkl')

# Optionally, remove the temporary model.pkl file
import os
os.remove('/kaggle/working/model.pkl')


In [34]:
from IPython.display import FileLink

# Create a link to the file
FileLink(r'/kaggle/working/gpt2model.zip')

/kaggle/working/gpt2model.zip